Predicting Urban Events with scikit-learn

In [ ]:
Step 1: Load Data

In [ ]:
Objective

1. Load and explore the data for different US cities to identify any data quality issues.
2. Prepare a cleaned dataset with duplicate entries removed for each city.

In [2]:
import pandas as pd
import numpy as np

In [4]:
# Load data from a Parquet file
safety_SanFrancisco_df = pd.read_parquet('safety-SanFrancisco.parquet')

In [5]:
safety_SanFrancisco_df.head()

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
0,1,Safety,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None,
1,2,Safety,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311,
2,3,Safety,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone,
3,7,Safety,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone,
4,10,Safety,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,None,200 Block of DALEWOOD WAY,37.737953,-122.456498,None,


In [6]:
safety_SanFrancisco_df

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
0,1,Safety,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None,
1,2,Safety,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311,
2,3,Safety,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone,
3,7,Safety,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone,
4,10,Safety,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,None,200 Block of DALEWOOD WAY,37.737953,-122.456498,None,
...,...,...,...,...,...,...,...,...,...,...,...,...
3958884,1203165,Safety,311_All,2016-12-13 07:11:27,Streetlights,Streetlight - Light_On_all_the_time,Open,Intersection of 13TH ST and SOUTH VAN NESS AVE,37.769814,-122.417755,Phone,
3958885,1203168,Safety,311_All,2017-08-20 09:53:26,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of WOOLSEY ST and HAMILTON ST,37.723687,-122.408529,Phone,
3958886,1203170,Safety,311_All,2018-06-06 10:51:00,General Request - PUBLIC WORKS,request_for_service,Closed,"240 SHOTWELL ST, SAN FRANCISCO, CA, 94110",37.764944,-122.416391,Mobile/Open311,
3958887,1203173,Safety,311_All,2019-03-19 14:14:18,Street and Sidewalk Cleaning,Bulky Items,Closed,"1293 WEBSTER ST, SAN FRANCISCO, CA, 94115",37.782656,-122.431373,Web,


#columns that contain constant values removed


In [7]:
# Identify constant columns
constant_columns = safety_SanFrancisco_df.columns[safety_SanFrancisco_df.nunique() <= 1]

In [8]:
constant_columns

Index(['dataType', 'extendedProperties'], dtype='object')

In [9]:
# Remove constant columns
safety_SanFrancisco_df = safety_SanFrancisco_df.drop(columns=constant_columns)

In [10]:
safety_SanFrancisco_df

,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source
0,1,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311
2,3,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone
3,7,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone
4,10,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,None,200 Block of DALEWOOD WAY,37.737953,-122.456498,None
...,...,...,...,...,...,...,...,...,...,...
3958884,1203165,311_All,2016-12-13 07:11:27,Streetlights,Streetlight - Light_On_all_the_time,Open,Intersection of 13TH ST and SOUTH VAN NESS AVE,37.769814,-122.417755,Phone
3958885,1203168,311_All,2017-08-20 09:53:26,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of WOOLSEY ST and HAMILTON ST,37.723687,-122.408529,Phone
3958886,1203170,311_All,2018-06-06 10:51:00,General Request - PUBLIC WORKS,request_for_service,Closed,"240 SHOTWELL ST, SAN FRANCISCO, CA, 94110",37.764944,-122.416391,Mobile/Open311
3958887,1203173,311_All,2019-03-19 14:14:18,Street and Sidewalk Cleaning,Bulky Items,Closed,"1293 WEBSTER ST, SAN FRANCISCO, CA, 94115",37.782656,-122.431373,Web


#date columns converted to the datetime type

In [14]:
safety_SanFrancisco_df['dateTime'] =pd.to_datetime(safety_SanFrancisco_df['dateTime'])

In [15]:
safety_SanFrancisco_df


,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source
0,1,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311
2,3,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone
3,7,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone
4,10,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,None,200 Block of DALEWOOD WAY,37.737953,-122.456498,None
...,...,...,...,...,...,...,...,...,...,...
3958884,1203165,311_All,2016-12-13 07:11:27,Streetlights,Streetlight - Light_On_all_the_time,Open,Intersection of 13TH ST and SOUTH VAN NESS AVE,37.769814,-122.417755,Phone
3958885,1203168,311_All,2017-08-20 09:53:26,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of WOOLSEY ST and HAMILTON ST,37.723687,-122.408529,Phone
3958886,1203170,311_All,2018-06-06 10:51:00,General Request - PUBLIC WORKS,request_for_service,Closed,"240 SHOTWELL ST, SAN FRANCISCO, CA, 94110",37.764944,-122.416391,Mobile/Open311
3958887,1203173,311_All,2019-03-19 14:14:18,Street and Sidewalk Cleaning,Bulky Items,Closed,"1293 WEBSTER ST, SAN FRANCISCO, CA, 94115",37.782656,-122.431373,Web


#categorical columns (that have less than a few hundred unique string values) converted to categorical data type, in order to use less memory.

In [16]:
def optimize_memory(df, max_unique_values=200):
    """
    Converts string columns with less than max_unique_values unique values to 'category' data type.

    Args:
        df (pd.DataFrame): The input DataFrame.
        max_unique_values (int): The maximum number of unique values for a column to be converted to 'category'.

    Returns:
        pd.DataFrame: The DataFrame with optimized data types.
    """
    for col in df.select_dtypes(include='object').columns:
        if len(df[col].unique()) < max_unique_values:
            df[col] = df[col].astype('category')
    return df

In [17]:
safety_SanFrancisco_df=optimize_memory(safety_SanFrancisco_df)

In [18]:
safety_SanFrancisco_df

,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source
0,1,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,NaN,ELM ST/FRANKLIN ST,37.781286,-122.422187,NaN
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311
2,3,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone
3,7,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone
4,10,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,NaN,200 Block of DALEWOOD WAY,37.737953,-122.456498,NaN
...,...,...,...,...,...,...,...,...,...,...
3958884,1203165,311_All,2016-12-13 07:11:27,Streetlights,Streetlight - Light_On_all_the_time,Open,Intersection of 13TH ST and SOUTH VAN NESS AVE,37.769814,-122.417755,Phone
3958885,1203168,311_All,2017-08-20 09:53:26,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of WOOLSEY ST and HAMILTON ST,37.723687,-122.408529,Phone
3958886,1203170,311_All,2018-06-06 10:51:00,General Request - PUBLIC WORKS,request_for_service,Closed,"240 SHOTWELL ST, SAN FRANCISCO, CA, 94110",37.764944,-122.416391,Mobile/Open311
3958887,1203173,311_All,2019-03-19 14:14:18,Street and Sidewalk Cleaning,Bulky Items,Closed,"1293 WEBSTER ST, SAN FRANCISCO, CA, 94115",37.782656,-122.431373,Web


#rows with identical values removed. This can be achieved with the DataFrame.drop_duplicates method. It is recommended to write a Python function with these steps which will be re-used later.


In [19]:
def remove_duplicate_rows(df, subset=None, keep='first', inplace=False):
    """
    Removes duplicate rows from a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to remove duplicates from.
        subset (list or str, optional): Columns to consider for identifying duplicates. 
            If None, all columns are used. Defaults to None.
        keep ({'first', 'last', False}, optional): How to handle duplicates.
            - 'first' (default): Drop duplicates except for the first occurrence.
            - 'last': Drop duplicates except for the last occurrence.
            - False: Drop all duplicates.
        inplace (bool, optional): Whether to modify the DataFrame in place. 
            Defaults to False.

    Returns:
        pd.DataFrame or None: DataFrame with duplicates removed or None if inplace=True.
    """
    return df.drop_duplicates(subset=subset, keep=keep, inplace=inplace)

In [20]:
safety_SanFrancisco_df=remove_duplicate_rows(safety_SanFrancisco_df)

In [21]:
safety_SanFrancisco_df

,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source
0,1,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,NaN,ELM ST/FRANKLIN ST,37.781286,-122.422187,NaN
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311
2,3,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone
3,7,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone
4,10,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,NaN,200 Block of DALEWOOD WAY,37.737953,-122.456498,NaN
...,...,...,...,...,...,...,...,...,...,...
3958884,1203165,311_All,2016-12-13 07:11:27,Streetlights,Streetlight - Light_On_all_the_time,Open,Intersection of 13TH ST and SOUTH VAN NESS AVE,37.769814,-122.417755,Phone
3958885,1203168,311_All,2017-08-20 09:53:26,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of WOOLSEY ST and HAMILTON ST,37.723687,-122.408529,Phone
3958886,1203170,311_All,2018-06-06 10:51:00,General Request - PUBLIC WORKS,request_for_service,Closed,"240 SHOTWELL ST, SAN FRANCISCO, CA, 94110",37.764944,-122.416391,Mobile/Open311
3958887,1203173,311_All,2019-03-19 14:14:18,Street and Sidewalk Cleaning,Bulky Items,Closed,"1293 WEBSTER ST, SAN FRANCISCO, CA, 94115",37.782656,-122.431373,Web


#Store the cleaned dataset to disk in parquet format as safety-SanFrancisco-1.parquet

In [22]:
safety_SanFrancisco_df.to_parquet('safety-SanFrancisco-1.parquet')